In [1]:
# importing necessary libraries
import numpy as np
import pandas as pd
import nltk 
import matplotlib.pyplot as plt

In [2]:
datatrain = pd.read_csv("D:/Spyder/Data-Envelopment-Analysis-Excel/Data/Disaster/train.csv")
datatest = pd.read_csv("D:/Spyder/Data-Envelopment-Analysis-Excel/Data/Disaster/test.csv")

In [3]:
datatrain.head()

,id,keyword,location,text,target
0,1,NaN,NaN,Our Deeds are the Reason of this #earthquake M...,1
1,4,NaN,NaN,Forest fire near La Ronge Sask. Canada,1
2,5,NaN,NaN,All residents asked to 'shelter in place' are ...,1
3,6,NaN,NaN,"13,000 people receive #wildfires evacuation or...",1
4,7,NaN,NaN,Just got sent this photo from Ruby #Alaska as ...,1


In [4]:
datatest.head()

,id,keyword,location,text
0,0,NaN,NaN,Just happened a terrible car crash
1,2,NaN,NaN,"Heard about #earthquake is different cities, s..."
2,3,NaN,NaN,"there is a forest fire at spot pond, geese are..."
3,9,NaN,NaN,Apocalypse lighting. #Spokane #wildfires
4,11,NaN,NaN,Typhoon Soudelor kills 28 in China and Taiwan


In [5]:
def basic_info(data):
    print(data.shape)
    print(data.columns)
    print(data.info())

In [6]:
basic_info(datatest)

(3263, 4)
Index(['id', 'keyword', 'location', 'text'], dtype='object')
<class 'pandas.core.frame.DataFrame'>
RangeIndex: 3263 entries, 0 to 3262
Data columns (total 4 columns):
 #   Column    Non-Null Count  Dtype 
---  ------    --------------  ----- 
 0   id        3263 non-null   int64 
 1   keyword   3237 non-null   object
 2   location  2158 non-null   object
 3   text      3263 non-null   object
dtypes: int64(1), object(3)
memory usage: 102.1+ KB
None


In [7]:
basic_info(datatrain)

(7613, 5)
Index(['id', 'keyword', 'location', 'text', 'target'], dtype='object')
<class 'pandas.core.frame.DataFrame'>
RangeIndex: 7613 entries, 0 to 7612
Data columns (total 5 columns):
 #   Column    Non-Null Count  Dtype 
---  ------    --------------  ----- 
 0   id        7613 non-null   int64 
 1   keyword   7552 non-null   object
 2   location  5080 non-null   object
 3   text      7613 non-null   object
 4   target    7613 non-null   int64 
dtypes: int64(2), object(3)
memory usage: 297.5+ KB
None


In [8]:
from nltk.stem import PorterStemmer
ps = PorterStemmer()

In [9]:
import string
import re
remove_hash = re.compile("#")
stopwords = nltk.corpus.stopwords.words('english')
def preprocessing_and_cleaining(x):
    x = x.lower()
    #print(x)
    x = remove_hash.sub("", x)
    x = x.split(" ")
    x = [i for i in x if i not in stopwords]
    #print(x)
    x = [i for i in x if i not in string.punctuation]
    #print(x)
    x = " ".join([ps.stem(i) for i in x])
    #print(x)
    return x

In [10]:
#cleaning the train and test tweets
datatrain['Cleaned_tweet'] = datatrain['text'].apply(lambda x: preprocessing_and_cleaining(x))

In [11]:
datatrain.head()

,id,keyword,location,text,target,Cleaned_tweet
0,1,NaN,NaN,Our Deeds are the Reason of this #earthquake M...,1,deed reason earthquak may allah forgiv us
1,4,NaN,NaN,Forest fire near La Ronge Sask. Canada,1,forest fire near la rong sask. canada
2,5,NaN,NaN,All residents asked to 'shelter in place' are ...,1,resid ask 'shelter place' notifi officers. eva...
3,6,NaN,NaN,"13,000 people receive #wildfires evacuation or...",1,"13,000 peopl receiv wildfir evacu order califo..."
4,7,NaN,NaN,Just got sent this photo from Ruby #Alaska as ...,1,got sent photo rubi alaska smoke wildfir pour ...


In [12]:
datatest['Cleaned_tweet'] = datatest['text'].apply(lambda x: preprocessing_and_cleaining(x))

In [13]:
datatest.head()

,id,keyword,location,text,Cleaned_tweet
0,0,NaN,NaN,Just happened a terrible car crash,happen terribl car crash
1,2,NaN,NaN,"Heard about #earthquake is different cities, s...","heard earthquak differ cities, stay safe every..."
2,3,NaN,NaN,"there is a forest fire at spot pond, geese are...","forest fire spot pond, gees flee across street..."
3,9,NaN,NaN,Apocalypse lighting. #Spokane #wildfires,apocalyps lighting. spokan wildfir
4,11,NaN,NaN,Typhoon Soudelor kills 28 in China and Taiwan,typhoon soudelor kill 28 china taiwan


Let's use TF-IDF for vectorization and make first out Logistic Regression

In [14]:
from sklearn.feature_extraction.text import TfidfVectorizer
from sklearn.linear_model import LogisticRegression
from sklearn.metrics import accuracy_score
from sklearn.model_selection import train_test_split

In [15]:
tfidf = TfidfVectorizer()
tfidf.fit(datatrain['Cleaned_tweet'])

TfidfVectorizer(analyzer='word', binary=False, decode_error='strict',
                dtype=<class 'numpy.float64'>, encoding='utf-8',
                input='content', lowercase=True, max_df=1.0, max_features=None,
                min_df=1, ngram_range=(1, 1), norm='l2', preprocessor=None,
                smooth_idf=True, stop_words=None, strip_accents=None,
                sublinear_tf=False, token_pattern='(?u)\\b\\w\\w+\\b',
                tokenizer=None, use_idf=True, vocabulary=None)

In [16]:
X = tfidf.transform(datatrain['Cleaned_tweet'])
X_test = tfidf.transform(datatest['Cleaned_tweet'])

In [17]:
X_train, X_val, y_train, y_val = train_test_split(X, datatrain['target'], train_size = 0.75, random_state = 0)

In [18]:
for c in [0.01, 0.05, 0.25, 0.5, 0.75, 1]:
    lr = LogisticRegression(C=c)
    lr.fit(X_train, y_train)
    print("Accuracy for C = %s: %s" % (c, accuracy_score(y_val, lr.predict(X_val))))

Accuracy for C = 0.01: 0.5814075630252101
Accuracy for C = 0.05: 0.6638655462184874
Accuracy for C = 0.25: 0.7746848739495799
Accuracy for C = 0.5: 0.7983193277310925
Accuracy for C = 0.75: 0.8072478991596639
Accuracy for C = 1: 0.8125


taking C = 1

In [25]:
final_tfidf = LogisticRegression(C = 1, max_iter=5000)
final_tfidf.fit(X, datatrain['target'])
#print("Final Accuracy: %s" %accuracy_score(datatrain['target'], final_tfidf.predict(X_test)))
prediction_df = pd.DataFrame({"id": datatest['id'],"Prediction": final_tfidf.predict(X_test)})

In [26]:
prediction_df

,id,Prediction
0,0,1
1,2,0
2,3,1
3,9,1
4,11,1
...,...,...
3258,10861,1
3259,10865,0
3260,10868,1
3261,10874,1


Let's make a random forest for classification

In [33]:
from sklearn.ensemble import RandomForestClassifier

In [34]:
rfc = RandomForestClassifier(random_state=0)

In [41]:
for max_features in ["auto", "sqrt", "log2"]:
    for obb in [True, False]:
        rfc_trying = RandomForestClassifier(max_features= max_features, random_state=1, oob_score = obb)
        rfc_trying.fit(X_train, y_train)
        print("Accuracy for {0} for obb {1}".format(accuracy_score(y_val, lr.predict(X_val)), str(obb)))

Accuracy for 0.8125 for obb True
Accuracy for 0.8125 for obb False
Accuracy for 0.8125 for obb True
Accuracy for 0.8125 for obb False
Accuracy for 0.8125 for obb True
Accuracy for 0.8125 for obb False


In [36]:
rfc.fit(X_train, y_train)

RandomForestClassifier(bootstrap=True, ccp_alpha=0.0, class_weight=None,
                       criterion='gini', max_depth=None, max_features='auto',
                       max_leaf_nodes=None, max_samples=None,
                       min_impurity_decrease=0.0, min_impurity_split=None,
                       min_samples_leaf=1, min_samples_split=2,
                       min_weight_fraction_leaf=0.0, n_estimators=100,
                       n_jobs=None, oob_score=False, random_state=0, verbose=0,
                       warm_start=False)

In [37]:
accuracy_score(y_val, lr.predict(X_val))

0.8125

In [38]:
predictionrfr_df = pd.DataFrame({"id": datatest['id'],"Prediction": rfc.predict(X_test)})

In [39]:
predictionrfr_df

,id,Prediction
0,0,1
1,2,0
2,3,1
3,9,1
4,11,1
...,...,...
3258,10861,0
3259,10865,1
3260,10868,1
3261,10874,1


Lets try making SVM

In [42]:
from sklearn.svm import SVC

In [43]:
for c in [0.01, 0.05, 0.10, 0.25, 0.5, 0.75, 0.90, 1]:
    svcc = SVC(C=c)
    svcc.fit(X_train, y_train)
    print("Accuracy for C = %s: %s" % (c, accuracy_score(y_val, svcc.predict(X_val))))

Accuracy for C = 0.01: 0.5814075630252101
Accuracy for C = 0.05: 0.5814075630252101
Accuracy for C = 0.1: 0.601890756302521
Accuracy for C = 0.25: 0.694327731092437
Accuracy for C = 0.5: 0.7715336134453782
Accuracy for C = 0.75: 0.7951680672268907
Accuracy for C = 0.9: 0.8014705882352942
Accuracy for C = 1: 0.8061974789915967
